In [65]:
import pandas as pd
import openpyxl
from IPython.display import display
import xlsxwriter
import numpy as np

In [66]:
#指定月份
month = '202504'
#指定时间范围
start_date = pd.Timestamp('2025-01-01')
end_date = pd.Timestamp('2025-04-30')
df = pd.read_excel(fr"D:\000物料报表\{month}\单物料产值\物料精简原始数据5月.XLSX")
df

,物料编码,创建日期,是否冻结,图号,产品组,是否虚拟物料,国内/海外,物料描述
0,1101000100410,2018-07-10,NaN,1-FAA-A106,Z1,NaN,国内,线夹(CXW-128-Q2)
1,1101000100470,2018-07-10,NaN,1-FAA-A221,Z1,NaN,国内,网圈(CXW-128-Q2)
2,1101000100480,2018-07-10,NaN,1-FAA-A222,Z1,NaN,国内,网罩内圈(CXW-128-Q2)
3,1101000100510,2018-07-10,NaN,1-FAA-A302,Z1,NaN,国内,左灯罩(CXW-128-Q2)
4,1101000100520,2018-07-10,NaN,1-FAA-A303,Z1,NaN,国内,右灯罩(CXW-128-Q2)
...,...,...,...,...,...,...,...,...
63771,112500040005A,2024-07-05,NaN,SAEG-N0011,F1,NaN,国内,附件包装盒_542*382*44_FJ-YKZP2
63772,112500040005B,2024-07-16,NaN,SAEG-N0011,F1,NaN,国内,附件包装盒_542*382*44_FJ-YKZP2
63773,112500040010A,2024-06-27,NaN,SAEF-A0010,F1,NaN,国内,蒸烤盘_银色_FJ-YKZP1
63774,112500040018A,2024-06-25,NaN,SAEH-A0010,F1,NaN,国内,蒸烤盘_银色_FJ-YKZP3


In [67]:
df = df.astype(str)

In [68]:
list_drop = []
df['删除原因'] = ''
for i in range(len(df)):
    if df.loc[i,'国内/海外'] == 10:
        list_drop.append(i)
        df.loc[i,'删除原因'] = '海外'
    if 'ZD70' in df.loc[i,'物料描述']:
        list_drop.append(i)
        df.loc[i,'删除原因'] = 'ODM'
    if df.loc[i,'图号'][:4] == 'LAFS':
        list_drop.append(i)
        df.loc[i,'删除原因'] = '火炬'
    if df.loc[i,'图号'][:4] == 'LAFT':
        list_drop.append(i)
        df.loc[i,'删除原因'] = '火炬'
    if df.loc[i,'物料编码'][:8] == '11010022':
        list_drop.append(i)
        df.loc[i,'删除原因'] = 'IMES'  
    if df.loc[i,'物料编码'][:8] == '11010027':
        list_drop.append(i)
        df.loc[i,'删除原因'] = '空调油烟机'  
    if df.loc[i,'图号'][:4] == 'PACA':
        list_drop.append(i)
        df.loc[i,'删除原因'] = '净存机'
    if df.loc[i,'产品组'] == 'ZN':
        list_drop.append(i)
        df.loc[i,'删除原因'] = '灶消'

In [69]:
df

,物料编码,创建日期,是否冻结,图号,产品组,是否虚拟物料,国内/海外,物料描述,删除原因
0,1101000100410,2018-07-10,nan,1-FAA-A106,Z1,nan,国内,线夹(CXW-128-Q2),
1,1101000100470,2018-07-10,nan,1-FAA-A221,Z1,nan,国内,网圈(CXW-128-Q2),
2,1101000100480,2018-07-10,nan,1-FAA-A222,Z1,nan,国内,网罩内圈(CXW-128-Q2),
3,1101000100510,2018-07-10,nan,1-FAA-A302,Z1,nan,国内,左灯罩(CXW-128-Q2),
4,1101000100520,2018-07-10,nan,1-FAA-A303,Z1,nan,国内,右灯罩(CXW-128-Q2),
...,...,...,...,...,...,...,...,...,...
63771,112500040005A,2024-07-05,nan,SAEG-N0011,F1,nan,国内,附件包装盒_542*382*44_FJ-YKZP2,
63772,112500040005B,2024-07-16,nan,SAEG-N0011,F1,nan,国内,附件包装盒_542*382*44_FJ-YKZP2,
63773,112500040010A,2024-06-27,nan,SAEF-A0010,F1,nan,国内,蒸烤盘_银色_FJ-YKZP1,
63774,112500040018A,2024-06-25,nan,SAEH-A0010,F1,nan,国内,蒸烤盘_银色_FJ-YKZP3,


In [70]:
len(list_drop)

1453

In [71]:
#依据产品组的产品组描述的对照关系，生成一列产品组描述
# 输入：字典, 产品组描述的对照关系，产品组描述的列名
product_dict = {
    'Z1': '油烟机',
    'ZZ': '不考核产品',
    'Z2': '灶具',
    'Z3': '消毒柜',
    'Z7': '热水器',
    'ZT': '热水器',
    'Z6': '烤箱',
    'Z4': '微波炉',
    'Z5': '蒸箱',  
    'Z8': '水槽洗碗机',
    'ZK': '蒸微',
    'ZL': '蒸烤烹饪机',
    'ZM': '灶集成',
    'ZP': '灶集成',
    'ZN': '灶集成',
    'ZQ': '蒸烤微烹饪机',
    'Y1': '不考核产品',
    'Y4': '灶烤烹饪机',
    'Z9': '不考核产品',
    'ZA': '不考核产品',
    'ZD': '家用净水机',
    'ZR': '不考核产品',
    'ZU': '不考核产品',
    'ZE': '不考核产品',
    'ZH': '不考核产品',
    'ZJ': '不考核产品',
    'ZS': '嵌入式洗碗机',
    'ZV': '不考核产品',
    'ZX': '不考核产品',
    'Y2': '不考核产品',
    'Y3': '不考核产品',
    'F1': '不考核产品'
}
df['产品组描述'] = df['产品组'].map(product_dict)


In [72]:
#依据产品组的国内海外的对照关系，生成一列国内/海外描述
# 输入：字典, 产品组描述的对照关系，产品组描述的列名
product_group_description_dict = {'20': '国内', '10': '海外','国内': '国内', '海外': '海外',20: '国内', 10: '海外','20.0': '国内', '10.0': '海外'}
df['国内/海外描述'] = df['国内/海外'].map(product_group_description_dict)
df


,物料编码,创建日期,是否冻结,图号,产品组,是否虚拟物料,国内/海外,物料描述,删除原因,产品组描述,国内/海外描述
0,1101000100410,2018-07-10,nan,1-FAA-A106,Z1,nan,国内,线夹(CXW-128-Q2),,油烟机,国内
1,1101000100470,2018-07-10,nan,1-FAA-A221,Z1,nan,国内,网圈(CXW-128-Q2),,油烟机,国内
2,1101000100480,2018-07-10,nan,1-FAA-A222,Z1,nan,国内,网罩内圈(CXW-128-Q2),,油烟机,国内
3,1101000100510,2018-07-10,nan,1-FAA-A302,Z1,nan,国内,左灯罩(CXW-128-Q2),,油烟机,国内
4,1101000100520,2018-07-10,nan,1-FAA-A303,Z1,nan,国内,右灯罩(CXW-128-Q2),,油烟机,国内
...,...,...,...,...,...,...,...,...,...,...,...
63771,112500040005A,2024-07-05,nan,SAEG-N0011,F1,nan,国内,附件包装盒_542*382*44_FJ-YKZP2,,不考核产品,国内
63772,112500040005B,2024-07-16,nan,SAEG-N0011,F1,nan,国内,附件包装盒_542*382*44_FJ-YKZP2,,不考核产品,国内
63773,112500040010A,2024-06-27,nan,SAEF-A0010,F1,nan,国内,蒸烤盘_银色_FJ-YKZP1,,不考核产品,国内
63774,112500040018A,2024-06-25,nan,SAEH-A0010,F1,nan,国内,蒸烤盘_银色_FJ-YKZP3,,不考核产品,国内


In [73]:
df.to_excel(fr"D:\000物料报表\{month}\单物料产值\物料精简原始数据处理稿.xlsx",index=False)

In [74]:
df = df.fillna('')
df

,物料编码,创建日期,是否冻结,图号,产品组,是否虚拟物料,国内/海外,物料描述,删除原因,产品组描述,国内/海外描述
0,1101000100410,2018-07-10,nan,1-FAA-A106,Z1,nan,国内,线夹(CXW-128-Q2),,油烟机,国内
1,1101000100470,2018-07-10,nan,1-FAA-A221,Z1,nan,国内,网圈(CXW-128-Q2),,油烟机,国内
2,1101000100480,2018-07-10,nan,1-FAA-A222,Z1,nan,国内,网罩内圈(CXW-128-Q2),,油烟机,国内
3,1101000100510,2018-07-10,nan,1-FAA-A302,Z1,nan,国内,左灯罩(CXW-128-Q2),,油烟机,国内
4,1101000100520,2018-07-10,nan,1-FAA-A303,Z1,nan,国内,右灯罩(CXW-128-Q2),,油烟机,国内
...,...,...,...,...,...,...,...,...,...,...,...
63771,112500040005A,2024-07-05,nan,SAEG-N0011,F1,nan,国内,附件包装盒_542*382*44_FJ-YKZP2,,不考核产品,国内
63772,112500040005B,2024-07-16,nan,SAEG-N0011,F1,nan,国内,附件包装盒_542*382*44_FJ-YKZP2,,不考核产品,国内
63773,112500040010A,2024-06-27,nan,SAEF-A0010,F1,nan,国内,蒸烤盘_银色_FJ-YKZP1,,不考核产品,国内
63774,112500040018A,2024-06-25,nan,SAEH-A0010,F1,nan,国内,蒸烤盘_银色_FJ-YKZP3,,不考核产品,国内


In [75]:
#删除掉删除原因中不为空的行,并且国内/海外描述值为国内的行
df_drop = df[(df['删除原因']=='') & (df['国内/海外描述'] == '国内')]
len(df_drop)

54094

In [76]:
# 在原始代码前添加日期转换（如果尚未转换）
df_drop['创建日期'] = pd.to_datetime(df_drop['创建日期'])

#按照产品组描述来对于df_drop进行分组计数，列名只留一个'产品组描述'和计数值'数量'
df_drop_group_all = df_drop[df_drop['创建日期'] <= end_date ].groupby(['产品组描述']).size().reset_index(name='数量')
df_drop_group_all

C:\Users\zhangbon\AppData\Local\Temp\ipykernel_13872\2269150744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['创建日期'] = pd.to_datetime(df_drop['创建日期'])


,产品组描述,数量
0,不考核产品,7731
1,家用净水机,2238
2,嵌入式洗碗机,3348
3,微波炉,624
4,水槽洗碗机,4306
5,油烟机,12819
6,消毒柜,2696
7,灶具,6468
8,灶集成,2164
9,烤箱,1386


In [77]:
#按照产品组名称来对于df_drop进行分组计数，创建日期范围为2024年1月到3月，列名只留一个'产品组描述'和计数值'数量'
df_drop_group_part = df_drop[(df_drop['创建日期'] >= start_date ) & (df_drop['创建日期'] <= end_date )].groupby(['产品组描述']).size().reset_index(name='数量')
df_drop_group_part

,产品组描述,数量
0,不考核产品,420
1,家用净水机,190
2,嵌入式洗碗机,405
3,微波炉,4
4,水槽洗碗机,219
5,油烟机,1198
6,消毒柜,103
7,灶具,284
8,灶集成,487
9,烤箱,7


In [78]:
#删除掉删除原因中不为空的行,并且国内/海外描述值为海外的行
df_haiwai =  df[(df['删除原因']=='') & (df['国内/海外描述'] == '海外')& (df['产品组描述'] != '不考核产品')]
print(len(df_haiwai))
df_haiwai['创建日期'] = pd.to_datetime(df_haiwai['创建日期'])
#创建日期小于当前的行数
df_haiwai_all = df_haiwai[df_haiwai['创建日期'] <= end_date]
len(df_haiwai_all)
print(f'海外物料总数量为：{len(df_haiwai_all)}')
#在当前范围内的海外物料数量
df_haiwai_part = df_haiwai[(df_haiwai['创建日期'] >= start_date) & (df_haiwai['创建日期'] <= end_date)]
len(df_haiwai_part)
print(f'规定范围内的海外物料数量为：{len(df_haiwai_part)}')

8175
海外物料总数量为：8041
规定范围内的海外物料数量为：556


C:\Users\zhangbon\AppData\Local\Temp\ipykernel_13872\1909754441.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_haiwai['创建日期'] = pd.to_datetime(df_haiwai['创建日期'])


In [79]:
#把df_drop_group_all和df_drop_group_part输出为同一个excel中
with pd.ExcelWriter(fr"D:\000物料报表\{month}\单物料产值\计数—国内.xlsx") as writer:
    df_drop_group_all.to_excel(writer, sheet_name='all', index=False)
    df_drop_group_part.to_excel(writer, sheet_name='part', index=False)